In [1]:
import json
from openai import OpenAI
from datasets import load_dataset
from config import OPEN_AI_TOKEN
from config import PROJECT_DIR
from general_utils.reader import JSONLineReader
from tqdm import tqdm
from general_utils.utils import parse_model_answer, get_openai_prediction

In [2]:
client = OpenAI(api_key=OPEN_AI_TOKEN)
fh = JSONLineReader()

In [3]:
dataset = load_dataset("lukasellinger/german_dpr_claim_verification_dissim-v1").get('train')

In [6]:
def get_prediction(claim, model="gpt-3.5-turbo"):
    response = client.chat.completions.create(
    model=model,
    temperature=0.1,
    logprobs=True,
    top_logprobs=5,
    messages=[
        {
            "role": "user",
            "content": f'Input: {claim} True or False?\nOutput:'
        }
    ],
    )
    txt_output = response.choices[0].message.content
    prediction = get_openai_prediction(response)
    if prediction == 'UNKOWN':
        prediction = parse_model_answer(txt_output)
    return txt_output, prediction

In [7]:
# Testing on a few examples
for entry in dataset.select(range(3)):
    word = entry.get('word')
    claim = f"{word}: {entry['claim']}"
    output, prediction = get_prediction(claim)
    print(f"WORD: {word}\nOVERVIEW: {claim}\n\nOUTPUT: {output}\nPREDICTION: {prediction}")
    print("\n\n----------------------------\n\n")

WORD: Liebe
OVERVIEW: Liebe: inniges Gefühl der Zuneigung für jemanden oder für etwas

OUTPUT: True
PREDICTIONS: SUPPORTED


----------------------------


WORD: Liebe
OVERVIEW: Liebe: sexuell oder erotisch motivierte Neigung zu jemandem oder zu einer Sache

OUTPUT: True
PREDICTIONS: SUPPORTED


----------------------------


WORD: Liebe
OVERVIEW: Liebe: Geschlechtsakt; Akt der körperlichen Vereinigung in Folge von 2

OUTPUT: False
PREDICTIONS: NOT_SUPPORTED


----------------------------




In [34]:
lines = []
for entry in tqdm(dataset):
    word = entry.get('word')
    claim = f"{word}: {entry['claim']}"
    generated_answer = get_prediction(claim, model='gpt-4o').lower()
    predicted = parse_model_answer(generated_answer)
    
    lines.append({
        'id': entry['id'],
        'word': entry['word'],
        'claim': claim,
        'predicted': predicted,
        'label': entry['label']
    })
        
fh.write('output_gpt4o.jsonl', lines)

100%|██████████| 168/168 [02:24<00:00,  1.17it/s]


# Create Task

## Zero Shot Task

In [78]:
# Creating an array of json tasks
model = "gpt-3.5-turbo"
tasks = []

for idx, entry in enumerate(dataset):
    word = entry['word']
    claim = f"{word}: {entry['claim']}"
    task = {
        "custom_id": f"task-{idx}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": model,
            "temperature": 0.1,
            "messages": [
                {
                    "role": "user",
                    "content": f'Input: {claim} True or False?\nOutput:'
                }
            ],
        }
    }
    
    tasks.append(task)

## Atomic Fact Task

In [4]:
from general_utils.atomic_facts import AtomicFactPromptGenerator

# Creating an array of json tasks
prompt_generator = AtomicFactPromptGenerator(demon_dir='../factscore/demos')

model = "gpt-3.5-turbo"
tasks = []

for idx, entry in enumerate(dataset):
    word = entry.get('english_word')
    claim = f"{word}: {entry['english_claim']}"
    messages = prompt_generator.create_prompt_from_sentence(claim)
    task = {
        "custom_id": f"task-{idx}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": model,
            "temperature": 0.1,
            "messages": messages,
        }
    }
    
    tasks.append(task)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lukasellinger/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
# Creating the file
identification = 'german_dpr_facts-gpt3_5-turbo'
file_name = PROJECT_DIR.joinpath(f'dataset/openai/batch_{identification}.jsonl')

with open(file_name, 'w') as file:
    for obj in tasks:
        file.write(json.dumps(obj) + '\n')

In [6]:
batch_file = client.files.create(
  file=open(file_name, "rb"),
  purpose="batch"
)

In [7]:
print(batch_file)

FileObject(id='file-ZLrSEvrONfVaWGwidT9vZY96', bytes=253871, created_at=1720534629, filename='batch_german_dpr_facts-gpt3_5-turbo.jsonl', object='file', purpose='batch', status='processed', status_details=None)


# Creating batch job

In [8]:
batch_job = client.batches.create(
  input_file_id=batch_file.id,
  endpoint="/v1/chat/completions",
  completion_window="24h"
)

In [12]:
batch_job = client.batches.retrieve(batch_job.id)
print(batch_job)

Batch(id='batch_QWlXIxtxZrVu7AwTwYWd8j1v', completion_window='24h', created_at=1720534637, endpoint='/v1/chat/completions', input_file_id='file-ZLrSEvrONfVaWGwidT9vZY96', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1720534935, error_file_id=None, errors=None, expired_at=None, expires_at=1720621037, failed_at=None, finalizing_at=1720534928, in_progress_at=1720534638, metadata=None, output_file_id='file-yDMqmxjw0Zp1n4hRiQVOFRp2', request_counts=BatchRequestCounts(completed=168, failed=0, total=168))


# Retrieving results

In [13]:
result_file_id = batch_job.output_file_id
result = client.files.content(result_file_id).content

In [14]:
result_file_name = f"batch_{identification}_results.jsonl"

with open(result_file_name, 'wb') as file:
    file.write(result)

In [15]:
# Loading data from saved file
results = fh.read(result_file_name)

In [90]:
lines = []
for res in results:
    task_id = res['custom_id']
    # Getting index from task id
    index = task_id.split('-')[-1]
    entry = dataset[int(index)]
    claim = entry['claim']
    word = entry['word']
    generated_answer = res['response']['body']['choices'][0]['message']['content'].lower()
    predicted = parse_model_answer(generated_answer)

    lines.append({
        'id': entry['id'],
        'word': entry['word'],
        'claim': claim,
        'predicted': predicted,
        'label': entry['label']
    })
    
fh.write(PROJECT_DIR.joinpath(f'dataset/openai/output_{identification}.jsonl', lines))

In [26]:
from general_utils.reader import JSONReader
import re

lines = {}
for res in results:
    task_id = res['custom_id']
    # Getting index from task id
    index = task_id.split('-')[-1]
    entry = dataset[int(index)]
    claim = entry['claim']
    word = entry['word']
    generated_answer = res['response']['body']['choices'][0]['message']['content']
    facts = []
    for line in generated_answer.split('\n'):
        fact_match = re.match(r'^\d+\.(.*)', line)
        if fact_match:
            facts.append(fact_match.group(1).strip())
        else:
            print(f'{entry["id"]} - {generated_answer}')
    lines[entry['id']] = facts

JSONReader().write(PROJECT_DIR.joinpath(f'dataset/openai/output_{identification}.json'), lines)